In [2]:
import os 
from kiwipiepy import Kiwi
import json
import re

KIWI = Kiwi(num_workers=2, load_default_dict=True, model_type='knlm')

SRC_PATH = ['./data/5_modern_essay', './data/5_modern_novel', './data/5_modern_poem']

PATTERN = [
    '같은', 
    '같이', 
    '처럼', 
    '와도', '과도', 
    '인 듯', '인듯', 
    '와같이', '와 같이', '과같이', '과 겉이',
    '와같은', '와 같은', '와같이', '와 같이'
]

# 파일 저장을 위해 토큰을 사전으로 변환
def token_to_dict(token) :
    return {'form':token[0], 'tag':token[1], 'start':token[2], 'len':token[3]}

# 동일내용 괄호 삭제
def remove_ss(sentence) :
    p = re.compile(r'[[|{|()}].+?[]|}|)]')

    result = ''
    m = re.search(p, sentence)
    while m is not None : 
        in_ss = sentence[m.start()+1:m.end()-1]
        front_ss = sentence[m.start()-len(in_ss):m.start()]
        if in_ss == front_ss :
            result = result + sentence[:m.start()]
        sentence = sentence[m.end():]
        m = re.search(p, sentence)
    result = result + sentence

    return result

# 보조관념 토큰 찾기
def find_aux_idea_tokens(pattern, sentence, tokens) : 
    result = []

    si = sentence.find(pattern) # 문장에서 패턴의 인덱스
    while si < len(sentence) :
        if si == -1 : # 패턴을 찾지 못한 경우
            break
        else : # 패턴을 찾은 경우
            ti = 0 # 토큰에서 패턴의 인덱스
            
            # 패턴 토큰 찾기
            while ti < len(tokens) :
                if tokens[ti][2] == si : break
                else : ti = ti + 1

            tag = tokens[ti-1][1] # 보조 관념의 형태소 태그
            if tag in ['NNG', 'NNP'] : # NNG : 일반 명사 / NNP : 고유 명사
                result.append(token_to_dict(tokens[ti-1]))

        # 하나의 패턴에 대해 복수의 표현이 있을수 있으므로 탐색 위치를 조정하고 추가 탐색
        si = sentence.find(pattern, si + len(pattern))

    return result


output = {}
sentence_count = 1
for i in range(0, len(SRC_PATH), 1) :
    # 경로와 파일이름 리스트 설정
    src_path = SRC_PATH[i]
    file_list = os.listdir(src_path)

    for j in range(0, len(file_list), 1) :
        file_name = file_list[j]
        book = None
        with open(src_path+'/'+file_name, 'r', encoding='utf-8') as file :
            book = json.load(file)

            for sentence in book['sentences'] :
                if len(sentence) > 1000 : break # 문장 분리가 제대로 되지 않은 경우

                for pattern in PATTERN : # 각 패턴마다 작업 수행
                    if pattern in sentence :
                        sentence = remove_ss(sentence) # 번역으로 인해 생긴 동알내용 괄호 삭제
                        tokens = KIWI.analyze(sentence)[0][0] # 형태소 분석
                        aux_tokens = find_aux_idea_tokens(pattern, sentence, tokens) # 패턴에 따른 보조관념 찾기

                        # 문장에 패턴에 대해 유효한 보조관념을 추출한 경우
                        if len(aux_tokens) != 0 : 
                            # json으로 쓰기위해 token을 사전형태로 변환
                            for k in range(0, len(tokens), 1) : 
                                tokens[k] = token_to_dict(tokens[k])

                            # 결과 사전에 추가
                            output[sentence_count] = {
                                'sentence': sentence,
                                'aux_tokens' : aux_tokens,
                                'tokens': tokens
                                
                            }
                            sentence_count = sentence_count + 1

# 파일로 저장
with open('./data/6_modern_data.json', 'w', encoding='utf-8') as file :
    json.dump(output, file, indent='\t', ensure_ascii=False)
